<a href="https://colab.research.google.com/github/maijathomas/MIS-2800/blob/main/IC_4_maija_thomas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IC (Feb 04) — Text Data in SQL (Student Version)

This in-class exercise practices the ideas from the Feb 04 slides:
- Text / character data and why it’s tricky
- `GROUP BY` + `COUNT()`
- `ORDER BY` (frequency vs alphabetical)
- Case sensitivity and `LOWER()` / `UPPER()`
- Spaces and `TRIM()`
- `LIKE` searches (and why they can be surprising)

**Instructions:** Run cells top → bottom. Write SQL inside the `query` blocks.


## 0) Setup (Run)
Creates a SQLite database `text_issues.db` and a table `product` with messy text.

In [1]:
!pip install --upgrade pandas ipython-sql prettytable==3.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.0 MB/s eta 0:00:00
  Attempting uninstall: prettytable
    Found existing installation: prettytable 3.17.0
    Uninstalling prettytable-3.17.0:
      Successfully uninstalled prettytable-3.17.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 3.0.0 which is incompatible.
gradio 5.50.0 requires pandas<3.0,>=1.0, but you have pandas 3.0.0 which is incompatible.
bqplot 0.12.45 requires pandas<3.0.0,>=1.0.0, but you have pandas 3.0.0 which is

In [2]:
# Install the SQL extension (not in sources, but required for magics)
!pip install ipython-sql
print("ipython sql installed")
# lines of code will run below for the installation

ipython sql installed


In [3]:
# Load the SQL magic extension
%load_ext sql

In [4]:
import sqlite3, pandas as pd

In [5]:
# This creates a brand-new, empty database file named 'text_issues.db'
conn = sqlite3.connect('text_issues.db')
print("database created")

database created


In [6]:
%sql sqlite:///text_issues.db

In [7]:
# create a table called inventory_table
%%sql
CREATE TABLE inventory_table (
  id INTEGER PRIMARY KEY,
  category TEXT,
  note TEXT
) STRICT;

 * sqlite:///text_issues.db
Done.


[]

In [8]:
# complet the table creation
%%sql
INSERT INTO inventory_table (id, category, note)
VALUES
  (1,  "Apple",   "fresh"),
  (2,  "apple",   "fresh"),
  (3,  "APPLE",   "fresh"),
  (4,  "Banana",  "ripe"),
  (5,  "banana",  "ripe"),
  (6,  "banana ", "ripe"),
  (7,  " banana", "ripe"),
  (8,  "to-do",   "punctuation"),
  (9,  "to–do",   "punctuation"),
  (10, "",        "empty string"),
  (11, 'None',      "NULL value");

 * sqlite:///text_issues.db
11 rows affected.


[]

In [9]:
%%sql
SELECT * FROM inventory_table;

 * sqlite:///text_issues.db
Done.


id,category,note
1,Apple,fresh
2,apple,fresh
3,APPLE,fresh
4,Banana,ripe
5,banana,ripe
6,banana,ripe
7,banana,ripe
8,to-do,punctuation
9,to–do,punctuation
10,,empty string


In [11]:
# Preview the data
run_sql("SELECT * FROM inventory_table;")

NameError: name 'run_sql' is not defined

## Part A — Quick Concept Check (We do it together)
1. Which values in `category` refer to the *same* category but look different?
2. What is the difference between an **empty string** (`''`) and **NULL**?
3. Why might punctuation cause matching problems?


# 1. Apple vs apple vs APPLE; Banana vs banana; to-do vs to do

# 2. null is a missing value vs empty string is empty text value

# 3. punctuation differences like to do and to-do sort them into different categories

## Part B1 — Grouping and counting (raw text)
Which product categories have the most products, and how many products are in each category (sorted from most to least)

> Add blockquote



In [ ]:
%%sql
-- Group by category and count.
SELECT category, COUNT(category)
FROM inventory_table
GROUP BY category
ORDER BY COUNT(category) DESC;

 * sqlite:///text_issues.db
Done.


category,COUNT(*)
to–do,1
to-do,1
banana,1
banana,1
apple,1
None,1
Banana,1
Apple,1
APPLE,1
banana,1


### Part B2 — Fix grouping with LOWER() - individual one

In [12]:
%%sql
-- Make category case-insensitive: group by LOWER(category) and count (ignore NULLs)
SELECT LOWER(category), COUNT(LOWER(category))
FROM inventory_table
WHERE category IS NOT NULL AND category <> ''
GROUP BY LOWER(category)
ORDER BY COUNT(LOWER(category)) DESC;

 * sqlite:///text_issues.db
Done.


LOWER(category),COUNT(LOWER(category))
apple,3
banana,2
to–do,1
to-do,1
none,1
banana,1
banana,1


## Part B3 — Fix spaces with TRIM() + LOWER()

In [13]:
%%sql

-- Clean spaces and case: group by LOWER(TRIM(category)) and count (ignore NULLs).
SELECT LOWER(TRIM(category)), COUNT(LOWER(TRIM(category)))
FROM inventory_table
WHERE category IS NOT NULL AND category <> ''
GROUP BY LOWER(TRIM(category))
ORDER BY COUNT(LOWER(TRIM(category))) DESC;

 * sqlite:///text_issues.db
Done.


LOWER(TRIM(category)),COUNT(LOWER(TRIM(category)))
banana,4
apple,3
to–do,1
to-do,1
none,1


## Part C1 — Order by category (alphabetical)

In [14]:
%%sql
-- Show unique cleaned categories in alphabetical order.
SELECT DISTINCT LOWER(TRIM(category))
FROM inventory_table
WHERE category IS NOT NULL AND category <> ''
ORDER BY LOWER(TRIM(category));

 * sqlite:///text_issues.db
Done.


LOWER(TRIM(category))
apple
banana
none
to-do
to–do


## Part D1 — Case matters

In [15]:
%%sql
-- Return rows where category = 'apple' (exact match).
SELECT * FROM inventory_table WHERE category = 'apple';

 * sqlite:///text_issues.db
Done.


id,category,note
2,apple,fresh


## Part D2 — Case-insensitive match

In [16]:
%%sql
-- Return rows that are apple regardless of case/spaces using LOWER(TRIM(category)) = 'apple'.
SELECT * FROM inventory_table WHERE LOWER(TRIM(category)) = 'apple';

 * sqlite:///text_issues.db
Done.


id,category,note
1,Apple,fresh
2,apple,fresh
3,APPLE,fresh


## Part D3 — Empty strings aren’t NULL

In [17]:
%%sql
-- Show rows where category is an empty string '' (NOT NULL).
SELECT * FROM inventory_table WHERE category = '';

 * sqlite:///text_issues.db
Done.


id,category,note
10,,empty string


## Part D4 — Find NULL categories

In [18]:
%%sql
-- Show rows where category IS NULL.
SELECT * FROM inventory_table WHERE category IS NULL;

 * sqlite:///text_issues.db
Done.


id,category,note
